# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [47]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [48]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,22.85,57,0,10.42,AU,1674555193
1,1,barrow,71.2906,-156.7887,-20.99,77,100,10.80,US,1674555327
2,2,vaini,-21.2000,-175.2000,27.09,89,40,3.09,TO,1674555327
3,3,kiama,-34.6833,150.8667,23.33,87,95,0.45,AU,1674555327
4,4,mangan,27.5167,88.5333,19.67,53,56,2.31,IN,1674555327
...,...,...,...,...,...,...,...,...,...,...
565,565,guatape,6.2327,-75.1586,13.47,95,98,0.97,CO,1674555538
566,566,pailon,-17.6500,-62.7500,23.82,91,100,3.62,BO,1674555539
567,567,buguda,19.8167,84.8000,31.65,22,5,2.58,IN,1674555539
568,568,westport,41.1415,-73.3579,2.99,70,100,4.63,US,1674555539


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [49]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(x='Lng', y='Lat', size = "Humidity", geo = True, tiles = "OSM", 
                                  frame_width = 800, frame_height = 600, color = "City")

# Display the map
hvplot.show(map)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [50]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[(city_data_df["Max Temp"] >= 15) & (city_data_df["Max Temp"] <= 27) &\
                            (city_data_df["Humidity"] >= 10) & (city_data_df["Humidity"] <= 50) &\
                            (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] <= 20)]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
103,103,najran,17.4924,44.1277,24.36,19,1,1.04,SA,1674555364
185,185,pokhara,28.2333,83.9833,24.00,18,1,2.77,NP,1674555400
230,230,taunggyi,20.7833,97.0333,24.50,22,13,1.27,MM,1674555419
263,263,tambacounda,13.7539,-13.7586,26.03,41,0,2.06,SN,1674555433
291,291,asyut,27.1810,31.1837,21.88,37,0,4.95,EG,1674555443
399,399,dali,25.7000,100.1833,15.83,29,12,2.93,CN,1674555478
514,514,umm lajj,25.0213,37.2685,23.81,50,0,3.45,SA,1674555520
519,519,tessalit,20.1986,1.0114,24.55,11,0,3.66,ML,1674555523


### Step 3: Create a new DataFrame called `hotel_df`.

In [51]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
103,najran,SA,17.4924,44.1277,19,
185,pokhara,NP,28.2333,83.9833,18,
230,taunggyi,MM,20.7833,97.0333,22,
263,tambacounda,SN,13.7539,-13.7586,41,
291,asyut,EG,27.1810,31.1837,37,
399,dali,CN,25.7000,100.1833,29,
514,umm lajj,SA,25.0213,37.2685,50,
519,tessalit,ML,20.1986,1.0114,11,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [52]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = np.nan
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
najran - nearest hotel: nan
pokhara - nearest hotel: Hotel Annapurna View Sarangkot
taunggyi - nearest hotel: Golden Island Cottages
tambacounda - nearest hotel: Le relais de Tamba
asyut - nearest hotel: فندق الحسين
dali - nearest hotel: 风花雪月大酒店
umm lajj - nearest hotel: nan
tessalit - nearest hotel: nan


,City,Country,Lat,Lng,Humidity,Hotel Name
103,najran,SA,17.4924,44.1277,19,NaN
185,pokhara,NP,28.2333,83.9833,18,Hotel Annapurna View Sarangkot
230,taunggyi,MM,20.7833,97.0333,22,Golden Island Cottages
263,tambacounda,SN,13.7539,-13.7586,41,Le relais de Tamba
291,asyut,EG,27.1810,31.1837,37,فندق الحسين
399,dali,CN,25.7000,100.1833,29,风花雪月大酒店
514,umm lajj,SA,25.0213,37.2685,50,NaN
519,tessalit,ML,20.1986,1.0114,11,NaN


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [54]:
%%capture --no-display

# Configure the map plot
vacay_map = hotel_df.hvplot.points(x='Lng', y='Lat', size = "Humidity", geo = True, tiles = "OSM", 
                              hover_cols=["Lng","Lat","City","Humidity","Hotel Name","Country"],
                              frame_width = 800, frame_height = 600, color = "City")

# Display the map
hvplot.show(vacay_map)